# 📊 Negotiation Database - 数据读取与初步分析

这个 notebook 用于读取和分析谈判数据库。

**使用前准备：**
1. 将下载的 `negotiations.db` 文件放在与此 notebook 相同的目录下
2. 确保安装了必要的库：`pip install pandas`

## 1️⃣ 导入必要的库

In [46]:
import sqlite3
import pandas as pd
import json
from datetime import datetime

## 2️⃣ 连接数据库

In [47]:
# 数据库文件路径（确保文件在同一目录下）
DB_PATH = "negotiations (1).db"

# 连接数据库
conn = sqlite3.connect(DB_PATH)
print("✅ 数据库连接成功！")

✅ 数据库连接成功！


## 3️⃣ 读取数据

In [48]:
# 读取所有 session 数据
df = pd.read_sql_query("SELECT * FROM negotiation_sessions", conn)

print(f"📊 总共有 {len(df)} 个谈判 session")
print(f"\n数据库包含以下字段：")
print(df.columns.tolist())

📊 总共有 58 个谈判 session

数据库包含以下字段：
['session_id', 'student_id', 'student_name', 'scenario_name', 'student_role', 'ai_role', 'ai_model', 'student_goes_first', 'use_memory', 'use_plan', 'total_rounds', 'transcript', 'ai_memory', 'ai_plan', 'student_deal_json', 'ai_deal_json', 'deal_reached', 'deal_failed', 'status', 'created_at', 'updated_at', 'feedback_text', 'feedback_generated_at', 'feedback_model']


## 4️⃣ 查看数据概览

In [49]:
# 显示关键字段
key_columns = [
    'session_id', 
    'student_name', 
    'scenario_name',  
    'total_rounds',
    'deal_reached', 
    'status',
    'created_at'
]

df_overview = df[key_columns].copy()

# 截短 session_id 方便查看
df_overview['session_id'] = df_overview['session_id'].str[:8] + '...'

print("\n📋 所有谈判 Session 概览：")
df_overview


📋 所有谈判 Session 概览：


,session_id,student_name,scenario_name,total_rounds,deal_reached,status,created_at
0,1144aa88...,Chris,Top_talent,6,0,completed,2025-11-15T07:14:40.297549
1,65194c01...,Chris,Top_talent,6,0,active,2025-11-15T08:55:59.726561
2,33b750dd...,Chris,Top_talent,6,0,active,2025-11-15T08:56:20.962549
3,0616b831...,Chris,Top_talent,6,1,completed,2025-11-15T07:51:27.472206
4,de114cde...,Chris,Top_talent,6,0,active,2025-11-15T09:00:20.503126
5,596c7714...,Chris,Top_talent,6,0,completed,2025-11-15T21:35:25.382179
6,ef312044...,Chris,Top_talent,6,0,active,2025-11-16T06:37:19.054443
7,77d701ac...,Chris,Top_talent,6,1,completed,2025-11-16T06:37:55.475751
8,a82b4baa...,Chris,Top_talent,6,1,completed,2025-11-17T08:02:12.298009
9,d356be90...,Chris,Top_talent,6,0,active,2025-11-17T19:01:18.399144


## 5️⃣ 基础统计信息

In [38]:
print("📈 基础统计：")
print(f"\n总谈判数: {len(df)}")
print(f"达成协议数: {df['deal_reached'].sum()}")
print(f"协议达成率: {df['deal_reached'].mean() * 100:.1f}%")
print(f"\n状态分布:")
print(df['status'].value_counts())
print(f"\n场景分布:")
print(df['scenario_name'].value_counts())

📈 基础统计：

总谈判数: 37
达成协议数: 6
协议达成率: 16.2%

状态分布:
status
active       29
completed     8
Name: count, dtype: int64

场景分布:
scenario_name
Top_talent    37
Name: count, dtype: int64


## 6️⃣ 查看具体谈判对话

In [52]:
# 选择第一个 session 查看详细对话
if len(df) > 0:
    session_index = 55  # 可以修改这个数字查看不同的 session
    
    session_id = df.iloc[session_index]['session_id']
    student_name = df.iloc[session_index]['student_name']
    scenario_name = df.iloc[session_index]['scenario_name']
    transcript_json = df.iloc[session_index]['transcript']
    
    print(f"\n📝 Session 详情")
    print(f"Session ID: {session_id}")
    print(f"Student: {student_name}")
    print(f"Scenario: {scenario_name}")
    print(f"\n{'='*80}")
    print(f"Transcript:")
    print(f"{'='*80}\n")
    
    # 解析并显示对话
    transcript = json.loads(transcript_json)
    for i, message in enumerate(transcript, 1):
        print(f"{i}. {message}\n")
else:
    print("⚠️ 数据库中没有数据")


📝 Session 详情
Session ID: 0cfa7ae0-b50b-4348-9f61-af6678ef3bcf
Student: Prof Cain
Scenario: Zlab_Split

Transcript:

1. Round 1.1 - Z-lab: I'm pleased to be negotiating with you, Dr. Pottier, regarding the exclusive license for your innovative acid reflux tablet. At Z-Lab, we’re excited about the potential of your compound and are fully committed to pursuing FDA approval to maximize its impact. To start the discussion, I’d like to propose a deal that reflects a fair balance of risk and reward given the significant investment and uncertainty involved in the FDA approval process.

I propose a new deal, let's call it **Deal P**, with the following terms:
- **Upfront Payment**: $15 million
- **Bonus if FDA Approval**: $10 million

This structure offers a substantial upfront payment to secure the license while also providing a meaningful bonus to reward the success of FDA approval. I believe this deal acknowledges the value of your invention and aligns our mutual interests in achieving regu

## 7️⃣ 查看达成的协议（如果有）

In [42]:
# 筛选达成协议的 session
deals = df[df['deal_reached'] == 1]

if len(deals) > 0:
    print(f"✅ 找到 {len(deals)} 个达成协议的谈判\n")
    
    for idx, row in deals.iterrows():
        print(f"{'='*80}")
        print(f"Session ID: {row['session_id'][:8]}...")
        print(f"学生: {row['student_name']}")
        print(f"场景: {row['scenario_name']}")
        
        # 显示协议内容
        if pd.notna(row['student_deal_json']):
            student_deal = json.loads(row['student_deal_json'])
            print(f"\nStudent 提出的协议:")
            for key, value in student_deal.items():
                print(f"  - {key}: {value}")
        
        if pd.notna(row['ai_deal_json']):
            ai_deal = json.loads(row['ai_deal_json'])
            print(f"\nAI 确认的协议:")
            for key, value in ai_deal.items():
                print(f"  - {key}: {value}")
        
        print()
else:
    print("❌ 没有达成协议的谈判")

✅ 找到 6 个达成协议的谈判

Session ID: 0616b831...
学生: Chris
场景: Top_talent

Student 提出的协议:
  - final_start_date: July 5th
  - final_work_location: New Haven, CT
  - final_base_salary: 135000
  - final_remote_work: 0 day(s) / week
  - final_signing_bonus: 5000
  - final_vacation_time: 2 weeks
  - final_blair_rotation: False

AI 确认的协议:
  - final_start_date: July 5th
  - final_work_location: New Haven, CT
  - final_base_salary: 135000
  - final_remote_work: 0 day(s) / week
  - final_signing_bonus: 5000
  - final_vacation_time: 2 weeks
  - final_blair_rotation: False
  - total_points_of_deal_to_me: 56000

Session ID: 77d701ac...
学生: Chris
场景: Top_talent

Student 提出的协议:
  - final_start_date: August 1st or later
  - final_work_location: New Haven, CT
  - final_base_salary: 132000
  - final_remote_work: 0 day(s) / week
  - final_signing_bonus: 30000
  - final_vacation_time: 2.5 weeks
  - final_blair_rotation: True

AI 确认的协议:
  - final_start_date: August 1st or later
  - final_work_location: New Haven,

## 8️⃣ 导出数据为 CSV（可选）

In [20]:
# 导出关键字段为 CSV
export_columns = [
    'session_id', 
    'student_id',
    'student_name', 
    'scenario_name',
    'student_role',
    'ai_role',
    'total_rounds',
    'deal_reached',
    'deal_failed',
    'status',
    'created_at',
    'updated_at'
]

df[export_columns].to_csv('negotiations_export.csv', index=False)
print("✅ 数据已导出为 negotiations_export.csv")

✅ 数据已导出为 negotiations_export.csv


## 9️⃣ 关闭数据库连接

In [21]:
conn.close()
print("✅ 数据库连接已关闭")

✅ 数据库连接已关闭


---

## 💡 后续分析建议

确认数据可以读取后，你可以进行更深入的分析：

1. **对话轮次分析**：平均谈判轮次、协议达成的轮次分布
2. **协议条款分析**：分析不同场景下达成的协议内容
3. **时间分析**：谈判持续时间、高峰时段
4. **学生表现**：不同学生的成功率、谈判策略

